# Notebook : Ingestion quotidienne de données éoliennes

## 📋 Objectif
Charger de manière incrémentale les données de production éolienne depuis GitHub vers le Lakehouse Bronze.

## 🔄 Logique
1. Identifier la date la plus récente dans Bronze
2. Calculer le jour suivant (date + 1)
3. Télécharger le fichier CSV correspondant depuis GitHub
4. Ajouter les nouvelles données en mode append

## ⚙️ Exécution
- **Fréquence recommandée** : Quotidienne (via pipeline)
- **Durée moyenne** : 50-60 secondes
- **Dépendances** : Repository GitHub public accessible

## 📦 Dépendances
- Lakehouse : LH_Wind_Power_Bronze
- Source : https://github.com/gsoulat/data-training-fabric/tree/main/eolienne

In [22]:
import requests
import pandas as pd
from datetime import datetime, timedelta

StatementMeta(, bd9e5d3e-9b73-4777-aef3-e1010efece40, 24, Finished, Available, Finished)

In [23]:
# Get the current notebook name
notebook_name = notebookutils.runtime.context.get("currentNotebookName")

# Get the current workspace name
workspace_name = notebookutils.runtime.context.get("currentWorkspaceName")

# Get the current workspace name
workspace_id = notebookutils.runtime.context.get("currentWorkspaceId")

display(notebook_name)
display(workspace_name)
display(workspace_id)

StatementMeta(, bd9e5d3e-9b73-4777-aef3-e1010efece40, 25, Finished, Available, Finished)

'NB_Get_Daily_Data_Python'

'My workspace'

'3e466a0e-52dd-454c-8eed-106445e4c9f1'

In [24]:
# URL de base du repository GitHub
base_url = "https://raw.githubusercontent.com/gsoulat/data-training-fabric/main/eolienne/"

# Chemin vers la table Bronze
# bronze_table_path = "Tables/dbo/wind_power"
bronze_table_path = "abfss://3e466a0e-52dd-454c-8eed-106445e4c9f1@onelake.dfs.fabric.microsoft.com/46f8ad23-c07e-4263-bf79-ae0ce1f4fed3/Tables/dbo/wind_power"

StatementMeta(, bd9e5d3e-9b73-4777-aef3-e1010efece40, 26, Finished, Available, Finished)

In [25]:
# ⚠️ CELLULE D'INITIALISATION - À exécuter UNE SEULE FOIS
# Cette cellule crée la table Bronze avec le premier fichier de données

# initial_date = "20240615"  # Premier jour disponible
# initial_url = f"{base_url}{initial_date}_wind_power_data.csv"

# print(f"📥 Téléchargement du fichier initial : {initial_url}")

# Télécharger et charger le premier fichier
# df_initial = pd.read_csv(initial_url)
# df_initial['date'] = pd.to_datetime(df_initial['date'])

# print(f"✅ Fichier chargé : {len(df_initial)} lignes")
# print(f"📊 Aperçu des données :")
# print(df_initial.head())

# Convertir en Spark DataFrame et sauvegarder
# df_spark_initial = spark.createDataFrame(df_initial)
# df_spark_initial.write.format('delta').mode("overwrite").save(bronze_table_path)

# print("✅ Table Bronze initialisée avec succès !")

StatementMeta(, bd9e5d3e-9b73-4777-aef3-e1010efece40, 27, Finished, Available, Finished)

In [26]:
# Vérifier que la table existe et afficher quelques statistiques
df_check = spark.read.format("delta").load(bronze_table_path)

print(f"📊 Nombre total de lignes : {df_check.count()}")
print(f"📅 Colonnes disponibles : {df_check.columns}")
print(f"🔍 Aperçu des 5 premières lignes :")
df_check.show(5)

StatementMeta(, bd9e5d3e-9b73-4777-aef3-e1010efece40, 28, Finished, Available, Finished)

📊 Nombre total de lignes : 1296
📅 Colonnes disponibles : ['production_id', 'date', 'time', 'turbine_name', 'capacity', 'location_name', 'latitude', 'longitude', 'region', 'status', 'responsible_department', 'wind_speed', 'wind_direction', 'energy_produced']
🔍 Aperçu des 5 premières lignes :
+-------------+-------------------+--------+------------+--------+-------------+--------+---------+--------+------+----------------------+----------+--------------+---------------+
|production_id|               date|    time|turbine_name|capacity|location_name|latitude|longitude|  region|status|responsible_department|wind_speed|wind_direction|energy_produced|
+-------------+-------------------+--------+------------+--------+-------------+--------+---------+--------+------+----------------------+----------+--------------+---------------+
|         6589|2024-06-16 00:00:00|06-00-00|   Turbine A|    2200|   Location 1| 34.0522|-118.2437|Region A|Online|            Operations|  19.98823|             W| 

In [27]:
# Charger les données existantes depuis Bronze
df_spark = spark.read.format("delta").load(bronze_table_path)

# Convertir en Pandas pour manipulation plus facile
df_pandas = df_spark.toPandas()

print(f"📊 Données actuelles dans Bronze : {len(df_pandas)} lignes")

StatementMeta(, bd9e5d3e-9b73-4777-aef3-e1010efece40, 29, Finished, Available, Finished)

📊 Données actuelles dans Bronze : 1296 lignes


In [28]:
# Trouver la date la plus récente dans les données
most_recent_date = pd.to_datetime(df_pandas['date'], format="%Y%m%d").max()

# Calculer le jour suivant
next_date = (most_recent_date + timedelta(days=1)).strftime("%Y%m%d")

print(f"📅 Date la plus récente : {most_recent_date.strftime('%Y-%m-%d')}")
print(f"➡️  Prochaine date à charger : {next_date}")

StatementMeta(, bd9e5d3e-9b73-4777-aef3-e1010efece40, 30, Finished, Available, Finished)

📅 Date la plus récente : 2024-06-17
➡️  Prochaine date à charger : 20240618


In [29]:
# Construire l'URL du fichier CSV
file_url = f"{base_url}{next_date}_wind_power_data.csv"
print(f"🌐 URL du fichier : {file_url}")

try:
    # Télécharger le CSV depuis GitHub
    df_pandas_new = pd.read_csv(file_url)
    
    # Convertir la colonne date en datetime
    df_pandas_new['date'] = pd.to_datetime(df_pandas_new['date'])
    
    print(f"✅ Nouvelles données téléchargées : {len(df_pandas_new)} lignes")
    print(f"📊 Aperçu des nouvelles données :")
    print(df_pandas_new.head())
    
except Exception as e:
    print(f"❌ Erreur lors du téléchargement : {e}")
    print(f"💡 Cela peut signifier que le fichier pour la date {next_date} n'existe pas encore.")

StatementMeta(, bd9e5d3e-9b73-4777-aef3-e1010efece40, 31, Finished, Available, Finished)

🌐 URL du fichier : https://raw.githubusercontent.com/gsoulat/data-training-fabric/main/eolienne/20240618_wind_power_data.csv
✅ Nouvelles données téléchargées : 432 lignes
📊 Aperçu des nouvelles données :
   production_id       date      time turbine_name  capacity location_name  \
0           7345 2024-06-18  00-00-00    Turbine A      2200    Location 1   
1           7346 2024-06-18  00-00-00    Turbine B      2000    Location 2   
2           7347 2024-06-18  00-00-00    Turbine C      2500    Location 3   
3           7348 2024-06-18  00-10-00    Turbine A      2200    Location 1   
4           7349 2024-06-18  00-10-00    Turbine B      2000    Location 2   

   latitude  longitude    region  status responsible_department  wind_speed  \
0   34.0522  -118.2437  Region A  Online             Operations    20.54374   
1   36.7783  -119.4179  Region B  Online             Operations    11.69426   
2   40.7128   -74.0060  Region C  Online             Operations    11.39494   
3   34.0522

In [30]:
# Convertir le DataFrame Pandas en Spark DataFrame
df_spark_new = spark.createDataFrame(df_pandas_new, schema=df_spark.schema)

# Ajouter les nouvelles données à la table Bronze (mode append)
df_spark_new.write.format('delta').mode("append").save(bronze_table_path)

print("✅ Données ajoutées avec succès dans le Lakehouse Bronze")
print(f"📊 Total après ajout : {spark.read.format('delta').load(bronze_table_path).count()} lignes")

StatementMeta(, bd9e5d3e-9b73-4777-aef3-e1010efece40, 32, Finished, Available, Finished)

✅ Données ajoutées avec succès dans le Lakehouse Bronze
📊 Total après ajout : 1728 lignes
